In [12]:
# Install necessary libraries if not already installed
# Note: Uncomment the next line if you encounter issues with imports
# !pip install google-cloud-vision google-auth google-auth-oauthlib google-api-python-client torch torchvision

# imports for auth, image processing, and PyTorch model
import requests
import base64
from google.cloud import vision
from google.oauth2 import service_account
from googleapiclient.discovery import build
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import io

# TODO -> put the path to your API key and service account file
api_key_file_path = '/Users/angelokaram/Desktop/Sac State Folders/Sac State Spring 2024/CSC 190/Senior-Project/private_keys.txt'
SERVICE_ACCOUNT_FILE = '/Users/angelokaram/Desktop/Sac State Folders/Sac State Spring 2024/CSC 190/Senior-Project/test-ai-sheet-1cbe18b0629d.json'

# Read the API key from the file
def get_api_key_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            if len(lines) >= 2:
                # API key on first line
                return lines[0].strip()
            else:
                raise ValueError("API key not found in the file.")
    except Exception as e:
        print(f"Failed to read the API key from file: {e}")
        return None

# Authentication and setup
API_KEY = get_api_key_from_file(api_key_file_path)
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=credentials)

# Spreadsheet details
SPREADSHEET_ID = '157Dpz8bSe0NMoUAcElBKG6InbRuiboWqht11-tdlnNA'
READ_RANGE_NAME = 'Sheet1!A2'  # Read Column A, Row 2
WRITE_RANGE_NAME = 'Sheet1!D2'  # Write to Column D, Row 2

# Preprocess the image (improves accuracy)
def preprocess_image(image_url):
    response = requests.get(image_url)
    img = Image.open(io.BytesIO(response.content)).convert('L')
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    return transform(img).unsqueeze(0)

# added CNN model training to handle different handwriting styles.
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 128 * 128, 10) 

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 32 * 128 * 128)
        x = torch.sigmoid(self.fc1(x))
        return x

# Image encoding for Google Vision API
def encode_image(image_url):
    """Fetches an image from the URL and encodes it in base64."""
    try:
        response = requests.get(image_url)
        response.raise_for_status()  # raise exception for HTTP errors
        return base64.b64encode(response.content).decode()
    except requests.RequestException as e:
        print(f"Failed to fetch image from URL: {e}")
        return None

# Detect text using Google Vision API
def detect_text_via_api(image_url):
    url = f"https://vision.googleapis.com/v1/images:annotate?key={API_KEY}"
    headers = {'Content-Type': 'application/json'}
    image_content = encode_image(image_url)
    
    body = {
        "requests": [
            {
                "image": {
                    "content": image_content
                },
                "features": [
                    {"type": "DOCUMENT_TEXT_DETECTION"}  # better type for handwritten text
                ]
            }
        ]
    }

    # API request and response handling
    response = requests.post(url, headers=headers, json=body)
    result = response.json()
    print(result)  # Debugging: prints the API response

    # Process the response to identify symbols and text
    if 'responses' in result and len(result['responses']) > 0 and 'textAnnotations' in result['responses'][0]:
        annotations = result['responses'][0]['textAnnotations']
        for annotation in annotations:
            description = annotation['description']
            print(description)  # Print out each detected text block for inspection
        return annotations[0]['description']
    else:
        return 'No text detected'

# Call functions to read image URL from Google Sheets and update with detected text
def read_and_update_sheet():
    # Read the image URL from the sheet
    result = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID, range=READ_RANGE_NAME).execute()
    image_url = result.get('values', [[None]])[0][0]
    if image_url:
        detected_text = detect_text_via_api(image_url)
        # Write what text was detected back to the sheet
        values = [[detected_text]]
        body = {'values': values}
        update_result = service.spreadsheets().values().update(
            spreadsheetId=SPREADSHEET_ID, range=WRITE_RANGE_NAME,
            valueInputOption='USER_ENTERED', body=body).execute()
        print(f'Cells updated: {update_result.get("updatedCells")}')
    else:
        print("No URL found in the specified cell.")

# Execute the main function
read_and_update_sheet()


{'responses': [{'textAnnotations': [{'locale': 'und', 'description': 'dog\nCat', 'boundingPoly': {'vertices': [{'x': 542, 'y': 51}, {'x': 671, 'y': 51}, {'x': 671, 'y': 185}, {'x': 542, 'y': 185}]}}, {'description': 'dog', 'boundingPoly': {'vertices': [{'x': 543, 'y': 55}, {'x': 667, 'y': 53}, {'x': 668, 'y': 140}, {'x': 544, 'y': 142}]}}, {'description': 'Cat', 'boundingPoly': {'vertices': [{'x': 560, 'y': 149}, {'x': 642, 'y': 145}, {'x': 643, 'y': 181}, {'x': 562, 'y': 185}]}}], 'fullTextAnnotation': {'pages': [{'width': 1286, 'height': 639, 'blocks': [{'boundingBox': {'vertices': [{'x': 542, 'y': 55}, {'x': 667, 'y': 51}, {'x': 671, 'y': 181}, {'x': 546, 'y': 185}]}, 'paragraphs': [{'boundingBox': {'vertices': [{'x': 543, 'y': 55}, {'x': 667, 'y': 53}, {'x': 668, 'y': 140}, {'x': 544, 'y': 142}]}, 'words': [{'boundingBox': {'vertices': [{'x': 543, 'y': 55}, {'x': 667, 'y': 53}, {'x': 668, 'y': 140}, {'x': 544, 'y': 142}]}, 'symbols': [{'boundingBox': {'vertices': [{'x': 543, 'y': 5